In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics roboflow supervision -q
!pip install git+https://github.com/roboflow/rf-detr.git -q
!pip install torch torchvision torchaudio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.

In [3]:
import os
import time
import pandas as pd
from ultralytics import YOLO

# --- Configuration ---
MODEL_PATH = "/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolov5nu.pt"
DATA_YAML = "/content/drive/MyDrive/Research/original LEMON images/data.yaml"
EXP_NAME = "5nYOLO"

EPOCHS = 50
BATCH = 16
LR = 0.0008
PATIENCE = 25

save_dir = f"/content/drive/MyDrive/Research/original LEMON images/{EXP_NAME}"
os.makedirs(save_dir, exist_ok=True)

# --- Resume Logic ---
resume_flag = False
last_weights_path = os.path.join(save_dir, "train", "weights", "last.pt")
if os.path.exists(last_weights_path):
    print(f"🔄 Resuming training from: {last_weights_path}")
    MODEL_PATH = last_weights_path
    resume_flag = True
else:
    print("🆕 Starting fresh training...")

# --- Load Model ---
model = YOLO(MODEL_PATH)

# --- Automatically detect backbone layers and freeze ---
print("⏸ Freezing backbone for initial training...")
backbone_layer_count = len(list(model.model.model.named_parameters())) // 2  # ~50% of layers as backbone
for idx, (name, param) in enumerate(model.model.model.named_parameters()):
    if idx < backbone_layer_count:
        param.requires_grad = False

# --- Stage 1: Train Detection Head Only ---
print("⏱ Stage 1: Training detection head only...")
model.train(
    data=DATA_YAML,
    epochs=10,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage1",
    exist_ok=True,
    resume=resume_flag,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

# --- Unfreeze Backbone ---
print("✅ Unfreezing backbone for full fine-tuning...")
for name, param in model.model.model.named_parameters():
    param.requires_grad = True

# --- Stage 2: Full Fine-tuning ---
print("⏱ Stage 2: Full model training with reduced mosaic near the end...")
train_start = time.time()

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage2",
    exist_ok=True,
    resume=False,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    close_mosaic=30,  # disable mosaic in last 20% of training
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

train_time = time.time() - train_start
print(f"✅ Training completed in {train_time / 60:.2f} minutes.")

# --- Evaluation with TTA ---
print("\n📊 Starting evaluation with TTA...")
metrics = model.val(augment=True)

# --- Calculate FPS ---
inference_ms = metrics.speed['inference']
fps = 1000 / inference_ms

# --- Get model size ---
model_file = model.ckpt_path if hasattr(model, 'ckpt_path') else MODEL_PATH
model_size_mb = os.path.getsize(model_file) / (1024 * 1024)

# --- Get parameter count ---
param_count = sum(p.numel() for p in model.model.parameters())

# --- Extract final metrics from results.csv ---
csv_file = os.path.join(results.save_dir, "results.csv")
df = pd.read_csv(csv_file)
final_metrics = df.iloc[-1].to_dict()

precision = final_metrics.get("metrics/precision(B)", 0)
recall = final_metrics.get("metrics/recall(B)", 0)
f1_score = (2 * precision * recall) / (precision + recall + 1e-6)

output_data = {
    "Model": os.path.basename(MODEL_PATH),
    "Precision": precision,
    "Recall": recall,
    "mAP50": final_metrics.get("metrics/mAP50(B)", 0),
    "mAP50-95": final_metrics.get("metrics/mAP50-95(B)", 0),
    "F1-score": f1_score,
    "FPS": fps,
    "Parameters": param_count,
    "Model Size (MB)": model_size_mb,
    "Training Time (min)": train_time / 60
}

results_csv_path = os.path.join(save_dir, "final_results.csv")
pd.DataFrame([output_data]).to_csv(results_csv_path, index=False)

print("\n✅ Final Metrics:")
for k, v in output_data.items():
    print(f"{k}: {v}")
print(f"\n📄 Saved to: {results_csv_path}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🆕 Starting fresh training...
⏸ Freezing backbone for initial training...
⏱ Stage 1: Training detection head only...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Research/original LEMON images/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, f

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1760  ultralytics.nn.modules.conv.Conv             [3, 16, 6, 2, 2]              
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      4800  ultralytics.nn.modules.block.C3              [32, 32, 1]                   
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     29184  ultralytics.nn.modules.block.C3              [64, 64, 2]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  7                  -1  1    295424  ultralytics

YOLOv5n summary: 153 layers, 2,508,659 parameters, 2,508,643 gradients, 7.2 GFLOPs

Transferred 391/427 items from pretrained weights
Freezing layer 'model.24.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 1.6±1.5 ms, read: 0.1±0.0 MB/s, size: 92.8 KB)


train: Scanning /content/drive/MyDrive/Research/original LEMON images/train/labels... 1211 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1211/1211 [14:16<00:00,  1.41it/s]


...


50 epochs completed in 0.451 hours.
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/5nYOLO/train_stage2/weights/last.pt, 5.3MB
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/5nYOLO/train_stage2/weights/best.pt, 5.3MB

Validating /content/drive/MyDrive/Research/original LEMON images/5nYOLO/train_stage2/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.10s/it]


                   all        111        600      0.839      0.773      0.855      0.614
Speed: 0.4ms preprocess, 3.7ms inference, 0.0ms loss, 9.5ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/5nYOLO/train_stage2
✅ Training completed in 27.36 minutes.

📊 Starting evaluation with TTA...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs
val: Fast image access ✅ (ping: 1.3±1.9 ms, read: 31.9±28.2 MB/s, size: 127.3 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.46it/s]


                   all        111        600      0.813      0.807      0.864      0.621
Speed: 3.9ms preprocess, 20.7ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/5nYOLO/train_stage2

✅ Final Metrics:
Model: yolov5nu.pt
Precision: 0.84014
Recall: 0.77082
mAP50: 0.85485
mAP50-95: 0.61337
F1-score: 0.8039880727093952
FPS: 48.32519727337805
Parameters: 2503139
Model Size (MB): 5.305060386657715
Training Time (min): 27.357507880528768

📄 Saved to: /content/drive/MyDrive/Research/original LEMON images/5nYOLO/final_results.csv


## **YOLO 8 N**

In [4]:
import os
import time
import pandas as pd
from ultralytics import YOLO

# --- Configuration ---
MODEL_PATH = "/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolov8n.pt"
DATA_YAML = "/content/drive/MyDrive/Research/original LEMON images/data.yaml"
EXP_NAME = "8nYOLO"

EPOCHS = 50
BATCH = 16
LR = 0.0008
PATIENCE = 25

save_dir = f"/content/drive/MyDrive/Research/original LEMON images/{EXP_NAME}"
os.makedirs(save_dir, exist_ok=True)

# --- Resume Logic ---
resume_flag = False
last_weights_path = os.path.join(save_dir, "train", "weights", "last.pt")
if os.path.exists(last_weights_path):
    print(f"🔄 Resuming training from: {last_weights_path}")
    MODEL_PATH = last_weights_path
    resume_flag = True
else:
    print("🆕 Starting fresh training...")

# --- Load Model ---
model = YOLO(MODEL_PATH)

# --- Automatically detect backbone layers and freeze ---
print("⏸ Freezing backbone for initial training...")
backbone_layer_count = len(list(model.model.model.named_parameters())) // 2  # ~50% of layers as backbone
for idx, (name, param) in enumerate(model.model.model.named_parameters()):
    if idx < backbone_layer_count:
        param.requires_grad = False

# --- Stage 1: Train Detection Head Only ---
print("⏱ Stage 1: Training detection head only...")
model.train(
    data=DATA_YAML,
    epochs=10,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage1",
    exist_ok=True,
    resume=resume_flag,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

# --- Unfreeze Backbone ---
print("✅ Unfreezing backbone for full fine-tuning...")
for name, param in model.model.model.named_parameters():
    param.requires_grad = True

# --- Stage 2: Full Fine-tuning ---
print("⏱ Stage 2: Full model training with reduced mosaic near the end...")
train_start = time.time()

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage2",
    exist_ok=True,
    resume=False,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    close_mosaic=30,  # disable mosaic in last 20% of training
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

train_time = time.time() - train_start
print(f"✅ Training completed in {train_time / 60:.2f} minutes.")

# --- Evaluation with TTA ---
print("\n📊 Starting evaluation with TTA...")
metrics = model.val(augment=True)

# --- Calculate FPS ---
inference_ms = metrics.speed['inference']
fps = 1000 / inference_ms

# --- Get model size ---
model_file = model.ckpt_path if hasattr(model, 'ckpt_path') else MODEL_PATH
model_size_mb = os.path.getsize(model_file) / (1024 * 1024)

# --- Get parameter count ---
param_count = sum(p.numel() for p in model.model.parameters())

# --- Extract final metrics from results.csv ---
csv_file = os.path.join(results.save_dir, "results.csv")
df = pd.read_csv(csv_file)
final_metrics = df.iloc[-1].to_dict()

precision = final_metrics.get("metrics/precision(B)", 0)
recall = final_metrics.get("metrics/recall(B)", 0)
f1_score = (2 * precision * recall) / (precision + recall + 1e-6)

output_data = {
    "Model": os.path.basename(MODEL_PATH),
    "Precision": precision,
    "Recall": recall,
    "mAP50": final_metrics.get("metrics/mAP50(B)", 0),
    "mAP50-95": final_metrics.get("metrics/mAP50-95(B)", 0),
    "F1-score": f1_score,
    "FPS": fps,
    "Parameters": param_count,
    "Model Size (MB)": model_size_mb,
    "Training Time (min)": train_time / 60
}

results_csv_path = os.path.join(save_dir, "final_results.csv")
pd.DataFrame([output_data]).to_csv(results_csv_path, index=False)

print("\n✅ Final Metrics:")
for k, v in output_data.items():
    print(f"{k}: {v}")
print(f"\n📄 Saved to: {results_csv_path}")


🆕 Starting fresh training...
⏸ Freezing backbone for initial training...
⏱ Stage 1: Training detection head only...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Research/original LEMON images/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolov8n.pt

train: Scanning /content/drive/MyDrive/Research/original LEMON images/train/labels.cache... 1211 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1211/1211 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 1.7±0.9 ms, read: 21.8±13.8 MB/s, size: 114.4 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]


...


50 epochs completed in 0.453 hours.
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/8nYOLO/train_stage2/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/8nYOLO/train_stage2/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Research/original LEMON images/8nYOLO/train_stage2/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.43it/s]


                   all        111        600      0.814      0.823      0.873      0.637
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 5.3ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/8nYOLO/train_stage2
✅ Training completed in 27.50 minutes.

📊 Starting evaluation with TTA...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.3 ms, read: 32.1±16.2 MB/s, size: 127.3 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  1.82it/s]


                   all        111        600      0.824      0.805      0.874      0.634
Speed: 5.8ms preprocess, 12.5ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/8nYOLO/train_stage2

✅ Final Metrics:
Model: yolov8n.pt
Precision: 0.81388
Recall: 0.82167
mAP50: 0.87296
mAP50-95: 0.63698
F1-score: 0.8177559484234976
FPS: 79.81665283603107
Parameters: 3005843
Model Size (MB): 6.246372222900391
Training Time (min): 27.498532513777416

📄 Saved to: /content/drive/MyDrive/Research/original LEMON images/8nYOLO/final_results.csv


# **YOLOV 12 N**

In [5]:
import os
import time
import pandas as pd
from ultralytics import YOLO

# --- Configuration ---
MODEL_PATH = "/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolo12n.pt"
DATA_YAML = "/content/drive/MyDrive/Research/original LEMON images/data.yaml"
EXP_NAME = "12nYOLO"

EPOCHS = 50
BATCH = 16
LR = 0.0008
PATIENCE = 25

save_dir = f"/content/drive/MyDrive/Research/original LEMON images/{EXP_NAME}"
os.makedirs(save_dir, exist_ok=True)

# --- Resume Logic ---
resume_flag = False
last_weights_path = os.path.join(save_dir, "train", "weights", "last.pt")
if os.path.exists(last_weights_path):
    print(f"🔄 Resuming training from: {last_weights_path}")
    MODEL_PATH = last_weights_path
    resume_flag = True
else:
    print("🆕 Starting fresh training...")

# --- Load Model ---
model = YOLO(MODEL_PATH)

# --- Automatically detect backbone layers and freeze ---
print("⏸ Freezing backbone for initial training...")
backbone_layer_count = len(list(model.model.model.named_parameters())) // 2  # ~50% of layers as backbone
for idx, (name, param) in enumerate(model.model.model.named_parameters()):
    if idx < backbone_layer_count:
        param.requires_grad = False

# --- Stage 1: Train Detection Head Only ---
print("⏱ Stage 1: Training detection head only...")
model.train(
    data=DATA_YAML,
    epochs=10,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage1",
    exist_ok=True,
    resume=resume_flag,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

# --- Unfreeze Backbone ---
print("✅ Unfreezing backbone for full fine-tuning...")
for name, param in model.model.model.named_parameters():
    param.requires_grad = True

# --- Stage 2: Full Fine-tuning ---
print("⏱ Stage 2: Full model training with reduced mosaic near the end...")
train_start = time.time()

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage2",
    exist_ok=True,
    resume=False,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    close_mosaic=30,  # disable mosaic in last 20% of training
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

train_time = time.time() - train_start
print(f"✅ Training completed in {train_time / 60:.2f} minutes.")

# --- Evaluation with TTA ---
print("\n📊 Starting evaluation with TTA...")
metrics = model.val(augment=True)

# --- Calculate FPS ---
inference_ms = metrics.speed['inference']
fps = 1000 / inference_ms

# --- Get model size ---
model_file = model.ckpt_path if hasattr(model, 'ckpt_path') else MODEL_PATH
model_size_mb = os.path.getsize(model_file) / (1024 * 1024)

# --- Get parameter count ---
param_count = sum(p.numel() for p in model.model.parameters())

# --- Extract final metrics from results.csv ---
csv_file = os.path.join(results.save_dir, "results.csv")
df = pd.read_csv(csv_file)
final_metrics = df.iloc[-1].to_dict()

precision = final_metrics.get("metrics/precision(B)", 0)
recall = final_metrics.get("metrics/recall(B)", 0)
f1_score = (2 * precision * recall) / (precision + recall + 1e-6)

output_data = {
    "Model": os.path.basename(MODEL_PATH),
    "Precision": precision,
    "Recall": recall,
    "mAP50": final_metrics.get("metrics/mAP50(B)", 0),
    "mAP50-95": final_metrics.get("metrics/mAP50-95(B)", 0),
    "F1-score": f1_score,
    "FPS": fps,
    "Parameters": param_count,
    "Model Size (MB)": model_size_mb,
    "Training Time (min)": train_time / 60
}

results_csv_path = os.path.join(save_dir, "final_results.csv")
pd.DataFrame([output_data]).to_csv(results_csv_path, index=False)

print("\n✅ Final Metrics:")
for k, v in output_data.items():
    print(f"{k}: {v}")
print(f"\n📄 Saved to: {results_csv_path}")


🆕 Starting fresh training...
⏸ Freezing backbone for initial training...
⏱ Stage 1: Training detection head only...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Research/original LEMON images/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolo12n.pt

train: Scanning /content/drive/MyDrive/Research/original LEMON images/train/labels.cache... 1211 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1211/1211 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 2.5±2.1 ms, read: 26.1±14.6 MB/s, size: 114.4 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]


...


50 epochs completed in 0.516 hours.
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/12nYOLO/train_stage2/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/12nYOLO/train_stage2/weights/best.pt, 5.5MB

Validating /content/drive/MyDrive/Research/original LEMON images/12nYOLO/train_stage2/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


                   all        111        600      0.848        0.8      0.871      0.638
Speed: 0.2ms preprocess, 7.9ms inference, 0.0ms loss, 5.5ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/12nYOLO/train_stage2
✅ Training completed in 31.21 minutes.

📊 Starting evaluation with TTA...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 6.4±12.9 ms, read: 45.4±30.8 MB/s, size: 127.3 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.27it/s]


                   all        111        600      0.836       0.79       0.87      0.645
Speed: 9.3ms preprocess, 20.5ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/12nYOLO/train_stage2

✅ Final Metrics:
Model: yolo12n.pt
Precision: 0.84779
Recall: 0.8
mAP50: 0.87153
mAP50-95: 0.63884
F1-score: 0.8232014861107994
FPS: 48.86163223486686
Parameters: 2556923
Model Size (MB): 5.335867881774902
Training Time (min): 31.210763490200044

📄 Saved to: /content/drive/MyDrive/Research/original LEMON images/12nYOLO/final_results.csv


# **yolo 11 m**

In [6]:
import os
import time
import pandas as pd
from ultralytics import YOLO

# --- Configuration ---
MODEL_PATH = "/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolo11m.pt"
DATA_YAML = "/content/drive/MyDrive/Research/original LEMON images/data.yaml"
EXP_NAME = "11mYOLO"

EPOCHS = 50
BATCH = 16
LR = 0.0008
PATIENCE = 25

save_dir = f"/content/drive/MyDrive/Research/original LEMON images/{EXP_NAME}"
os.makedirs(save_dir, exist_ok=True)

# --- Resume Logic ---
resume_flag = False
last_weights_path = os.path.join(save_dir, "train", "weights", "last.pt")
if os.path.exists(last_weights_path):
    print(f"🔄 Resuming training from: {last_weights_path}")
    MODEL_PATH = last_weights_path
    resume_flag = True
else:
    print("🆕 Starting fresh training...")

# --- Load Model ---
model = YOLO(MODEL_PATH)

# --- Automatically detect backbone layers and freeze ---
print("⏸ Freezing backbone for initial training...")
backbone_layer_count = len(list(model.model.model.named_parameters())) // 2  # ~50% of layers as backbone
for idx, (name, param) in enumerate(model.model.model.named_parameters()):
    if idx < backbone_layer_count:
        param.requires_grad = False

# --- Stage 1: Train Detection Head Only ---
print("⏱ Stage 1: Training detection head only...")
model.train(
    data=DATA_YAML,
    epochs=10,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage1",
    exist_ok=True,
    resume=resume_flag,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

# --- Unfreeze Backbone ---
print("✅ Unfreezing backbone for full fine-tuning...")
for name, param in model.model.model.named_parameters():
    param.requires_grad = True

# --- Stage 2: Full Fine-tuning ---
print("⏱ Stage 2: Full model training with reduced mosaic near the end...")
train_start = time.time()

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH,
    device=0,
    project=save_dir,
    name="train_stage2",
    exist_ok=True,
    resume=False,
    lr0=LR,
    patience=PATIENCE,
    optimizer="AdamW",
    weight_decay=0.0005,
    dropout=0.1,
    warmup_epochs=3,
    lrf=0.01,
    cos_lr=True,
    close_mosaic=30,  # disable mosaic in last 20% of training
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    scale=0.5,
    copy_paste=0.05
)

train_time = time.time() - train_start
print(f"✅ Training completed in {train_time / 60:.2f} minutes.")

# --- Evaluation with TTA ---
print("\n📊 Starting evaluation with TTA...")
metrics = model.val(augment=True)

# --- Calculate FPS ---
inference_ms = metrics.speed['inference']
fps = 1000 / inference_ms

# --- Get model size ---
model_file = model.ckpt_path if hasattr(model, 'ckpt_path') else MODEL_PATH
model_size_mb = os.path.getsize(model_file) / (1024 * 1024)

# --- Get parameter count ---
param_count = sum(p.numel() for p in model.model.parameters())

# --- Extract final metrics from results.csv ---
csv_file = os.path.join(results.save_dir, "results.csv")
df = pd.read_csv(csv_file)
final_metrics = df.iloc[-1].to_dict()

precision = final_metrics.get("metrics/precision(B)", 0)
recall = final_metrics.get("metrics/recall(B)", 0)
f1_score = (2 * precision * recall) / (precision + recall + 1e-6)

output_data = {
    "Model": os.path.basename(MODEL_PATH),
    "Precision": precision,
    "Recall": recall,
    "mAP50": final_metrics.get("metrics/mAP50(B)", 0),
    "mAP50-95": final_metrics.get("metrics/mAP50-95(B)", 0),
    "F1-score": f1_score,
    "FPS": fps,
    "Parameters": param_count,
    "Model Size (MB)": model_size_mb,
    "Training Time (min)": train_time / 60
}

results_csv_path = os.path.join(save_dir, "final_results.csv")
pd.DataFrame([output_data]).to_csv(results_csv_path, index=False)

print("\n✅ Final Metrics:")
for k, v in output_data.items():
    print(f"{k}: {v}")
print(f"\n📄 Saved to: {results_csv_path}")


🆕 Starting fresh training...
⏸ Freezing backbone for initial training...
⏱ Stage 1: Training detection head only...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Research/original LEMON images/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=/content/drive/MyDrive/Research/refence LEMONYOLO/Dataset/yolo11m.pt

train: Scanning /content/drive/MyDrive/Research/original LEMON images/train/labels.cache... 1211 images, 1 backgrounds, 0 corrupt: 100%|██████████| 1211/1211 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 2.3±3.4 ms, read: 20.7±21.7 MB/s, size: 114.4 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]


...


50 epochs completed in 0.722 hours.
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/11mYOLO/train_stage2/weights/last.pt, 40.5MB
Optimizer stripped from /content/drive/MyDrive/Research/original LEMON images/11mYOLO/train_stage2/weights/best.pt, 40.5MB

Validating /content/drive/MyDrive/Research/original LEMON images/11mYOLO/train_stage2/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]


                   all        111        600      0.841       0.82      0.882      0.654
Speed: 0.4ms preprocess, 13.0ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/11mYOLO/train_stage2
✅ Training completed in 43.71 minutes.

📊 Starting evaluation with TTA...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs
val: Fast image access ✅ (ping: 1.1±1.5 ms, read: 29.7±26.6 MB/s, size: 127.3 KB)


val: Scanning /content/drive/MyDrive/Research/original LEMON images/valid/labels.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:08<00:00,  1.24s/it]


                   all        111        600      0.835      0.818      0.886      0.665
Speed: 5.1ms preprocess, 57.2ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to /content/drive/MyDrive/Research/original LEMON images/11mYOLO/train_stage2

✅ Final Metrics:
Model: yolo11m.pt
Precision: 0.84086
Recall: 0.82
mAP50: 0.88206
mAP50-95: 0.65369
F1-score: 0.8302985018011743
FPS: 17.48779968106319
Parameters: 20030803
Model Size (MB): 38.799400329589844
Training Time (min): 43.70970667600632

📄 Saved to: /content/drive/MyDrive/Research/original LEMON images/11mYOLO/final_results.csv
